In [1]:
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

import pandas as pd

Using TensorFlow backend.


In [2]:
#Folder for the dataset
datasetFolder = '/home/carnd/dbpedia2016/all4_1x15/dataset/'

#Number of files
numberOfFiles = 2555

#Test split
testSplit=0.25

In [3]:
def load_data(datasetFolder, datasetXFile, datasetYFile):
    #print('Loading {} & {}'.format(datasetXFile, datasetYFile))
    # load file
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        cols = head.split(',')
        numberOfCols = len(cols)
        #print(numberOfCols)
        numberOfRows=0
        for line in f:
            numberOfRows+=1
        f.close()
    
    #print('{} x {}'.format(numberOfRows,numberOfCols))
    dataX = np.zeros([numberOfRows,numberOfCols-1],np.float32)
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        rowCounter=0
        for line in f:
            row=line.split(', ')
            for i in range(1, len(row)):
                if(int(row[i])<=0):
                    continue;
                dataX[rowCounter][int(row[i])-1]=1.0
            rowCounter+=1
        f.close()
        
    dataY=pd.read_csv(datasetFolder + datasetYFile)

    # delete the id fields
    #del dataX['id']
    del dataY['id']

    return dataX, dataY.as_matrix()

In [4]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv')

In [5]:
print(dataX.shape)
print(dataX[0:5])

(1204, 299760)
[[ 0.  1.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  1. ...,  0.  0.  0.]]


In [6]:
print(dataY.shape)
print(dataY[0:5])

(1204, 539)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [7]:
print("Input Features: {}".format(dataX.shape[1]))
print("Output Classes: {}".format(dataY.shape[1]))

Input Features: 299760
Output Classes: 539


In [8]:
# create model
# logisticRegression = Sequential(name='Simple Logistic Regression')
# logisticRegression.add(Dense(dataY.shape[1], input_dim=dataX.shape[1], activation='sigmoid', init='glorot_uniform'))

# simpleModel = Sequential(name='2 Fully Connected Layers')
# simpleModel.add(Dense(1024, input_dim=dataX.shape[1], activation='relu', init='glorot_uniform'))
# simpleModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_uniform'))

deepModel = Sequential(name='Deep Model (5 Dense Layers)')
deepModel.add(Dense(128, input_dim=dataX.shape[1], activation='relu', init='glorot_uniform'))
deepModel.add(Dense(1024, activation='relu', init='glorot_uniform'))
deepModel.add(Dense(768, activation='relu', init='glorot_uniform'))
deepModel.add(Dense(768, activation='relu', init='glorot_uniform'))
deepModel.add(Dense(768, activation='relu', init='glorot_uniform'))
deepModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_uniform'))

models = [deepModel]

In [9]:
# Compile model
import keras.backend as K

def count_predictions(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives, predicted_positives, possible_positives

def f1score(y_true, y_pred):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = 2.0 * precision * recall / (precision+recall+ K.epsilon())
    return f1score

def fBetaScore(y_true, y_pred, beta):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = (1+(beta*beta)) * precision * recall / ((beta*beta*precision)+recall+ K.epsilon())
    return f1score

for model in models:
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1score])

In [10]:
def fit_data(model, dataX, dataY):
    # Fit the model
    #model.fit(dataX, dataY, nb_epoch=5, verbose=2, batch_size=256)
    return model.train_on_batch(dataX, dataY)

In [11]:
def countPredictions(y_true, y_pred):
    true_positives = np.sum(np.round(y_pred*y_true))
    predicted_positives = np.sum(np.round(y_pred))
    possible_positives = np.sum(y_true)
    return true_positives, predicted_positives, possible_positives

In [12]:
#Randomize the list of numbers so we can split train and test dataset
listOfFiles=list(range(1,numberOfFiles+1))
import random
random.shuffle(listOfFiles)
splitIndex=int((1-testSplit)*numberOfFiles)

numberOfEons = 5
for eon in range(0, numberOfEons):
    print('\n***********************************\n {}/{}'.format(eon+1, numberOfEons))
    for trainIndex in range(0,splitIndex):
        print('==========================================')
        print('Learning for file {} / {} : datasetX/Y_{}...'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex]))
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[trainIndex]), 'datasetY_{}.csv'.format(listOfFiles[trainIndex]))
        for model in models:
            #print('Model = {}'.format(model.name))
            #model.fit(dataX, dataY, nb_epoch=1, verbose=1, batch_size=128)
            loss, f1score=fit_data(model,dataX, dataY)
            print('   Model = {} \t loss = {:.4f} \t f1-score = {:.4f}'.format(model.name, loss, f1score))
            
        
    counts = {} 
    for model in models:
        counts[model.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
    
    print('==========================================\n')
    for testIndex in range(splitIndex, numberOfFiles):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
        print ('Testing for file {} / {} : datasetX/Y_{}'.format(testIndex+1, numberOfFiles, listOfFiles[testIndex]))
        for model in models:
            predY=model.predict_on_batch(dataX)
            true_positives, predicted_positives, possible_positives = countPredictions(dataY, predY)
            counts[model.name]['true_positives'] += true_positives
            counts[model.name]['predicted_positives'] += predicted_positives
            counts[model.name]['possible_positives'] += possible_positives
    print('==========================================\n')
    
    
    print('{}.  {}/{} - Testing score:'.format(eon+1, eon+1, numberOfEons))
    for model in models:
        count = counts[model.name]
        precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
        recall = (count['true_positives'])/(count['possible_positives']+0.0001)
        f1score = 2.0 * precision * recall / (precision+recall+0.0001)
        print(' - Model = {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(model.name, f1score, precision, recall))
    print('==========================================\n')


***********************************
 1/5
Learning for file 1 / 1916 : datasetX/Y_1929...
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6931 	 f1-score = 0.0147
Learning for file 2 / 1916 : datasetX/Y_1875...
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6922 	 f1-score = 0.0138
Learning for file 3 / 1916 : datasetX/Y_594...
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6898 	 f1-score = 0.0144
Learning for file 4 / 1916 : datasetX/Y_955...
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6837 	 f1-score = 0.0170
Learning for file 5 / 1916 : datasetX/Y_1785...
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6688 	 f1-score = 0.0216
Learning for file 6 / 1916 : datasetX/Y_1281...
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6334 	 f1-score = 0.0297
Learning for file 7 / 1916 : datasetX/Y_1877...
   Model = Deep Model (5 Dense Layers) 	 loss = 0.5602 	 f1-score = 0.0464
Learning for file 8 / 1916 : datasetX/Y_2466...


ResourceExhaustedError: OOM when allocating tensor with shape[299760,128]
	 [[Node: mul_6 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](Variable_2/read, Variable_5/read)]]

Caused by op 'mul_6', defined at:
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in 

==================================================
# all4_1x15
==================================================
1. Eon 1/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.7228	 precision = 0.9145 	 recall = 0.5977
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8039	 precision = 0.9196 	 recall = 0.7142
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8867	 precision = 0.9334 	 recall = 0.8444
2. Eon 2/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.7795	 precision = 0.9135 	 recall = 0.6798
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8570	 precision = 0.9295 	 recall = 0.7950
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8983	 precision = 0.9330 	 recall = 0.8662
3. Eon 3/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.8022	 precision = 0.9159 	 recall = 0.7137
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8747	 precision = 0.9328 	 recall = 0.8236
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.9005	 precision = 0.9339 	 recall = 0.8694
4. Eon 4/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.8160	 precision = 0.9165 	 recall = 0.7354
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8826	 precision = 0.9342 	 recall = 0.8364
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.9014	 precision = 0.9339 	 recall = 0.8711
5. Eon 5/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.8251	 precision = 0.9173 	 recall = 0.7498
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8868	 precision = 0.9352 	 recall = 0.8432
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.9015	 precision = 0.9343 	 recall = 0.8709

## 